In [ ]:
import numpy as np

In [ ]:
dataset_name = '../data/domain_names_full.txt'
dataset = open(dataset_name, 'r').read().split()

In [ ]:
len(dataset)

In [ ]:
min(len(x) for x in dataset)

In [ ]:
np.argmax([len(x) for x in dataset])

In [ ]:
dataset[19484]

In [ ]:
for d in dataset[:10]:
    for c1, c2 in zip(d, d[1:]):
        print(c1,c2)

In [ ]:
'*' in set([y for x in dataset for y in x])

In [ ]:
# print the bigrams for each example
for d in dataset[:1]:
    example = ['*'] + list(d) + ['*']
    for c1, c2 in zip(example, example[1:]):
        print(c1,c2)

In [ ]:
# This step is expensive... 
bigrams = dict()
for d in dataset:
    example = ['*'] + list(d) + ['*']
    for c1, c2 in zip(example, example[1:]):
        bigrams[(c1, c2)] = bigrams.get((c1,c2), 0) + 1

In [ ]:
len(bigrams.keys())

In [ ]:
charset = ['*'] + sorted(list(set([y for x in dataset for y in x])))
ctoi = {c:i for i, c in enumerate(charset)}
itoc = {i:c for i, c in enumerate(charset)}

In [ ]:
ctoi['c']

In [ ]:
import numpy as np
bigram_count = np.zeros((5, 3, 4))

In [ ]:
bigram_count = np.zeros((len(charset), len(charset))).astype(int)

In [ ]:
bigram_count

In [ ]:
for d in dataset:  # <== remove [:10] to run for all examples
    example = ['*'] + list(d) + ['*']
    for c1, c2 in zip(example, example[1:]):
        bigram_count[ctoi[c1], ctoi[c2]] += 1

In [ ]:
bigram_count

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(bigram_count, cmap='Blues')
for i in range(len(charset)):
    for j in range(len(charset)):
        chars = itoc[i]+itoc[j]
        plt.text(i, j, chars,ha='center', va='bottom', color='grey', fontsize=8)
        plt.text(i, j, bigram_count[i,j],ha='center', va='top', color='grey', fontsize=8)
plt.axis('off')

In [ ]:
p = bigram_count[0]
p = p/p.sum()

In [ ]:
np.random.seed(42)

In [ ]:
draw = np.random.choice(charset, 1, p=p, replace=True)

In [ ]:
draw

In [ ]:
p.sum()

In [ ]:
def generate(bigram_count):
    new = []
    p = bigram_count[0]
    p = p/p.sum()
    draw = np.random.choice(charset, 1, p=p, replace=True)[0]
    while draw != '*':
        p = bigram_count[ctoi[draw]]
        p = p/p.sum()
        draw = np.random.choice(charset, 1, p=p, replace=True)[0]
        new.append(draw)
    return ''.join(new[:-1])

In [ ]:
for _ in range(10):
    print(generate(bigram_count))

### Negative Log Likelihood
Las probabilidades que el modelo asigna a cada bigrama individual debería ser lo más cercanas a 1 posible. Para cada bigrama, si la probabilidad fuera uniforme, tendríamos una probabilidad de `1/len(charset)`. Cualquier modelo que aprenda algo asignará a algunos bigramas valores más altos de probabilidad. Si nuestro modelo fuera perfecto, asignaría probabilidad 1 a cada uno de los bigramas individuales.

In [ ]:
1/len(charset)

El estimador de máxima verosimilitud (maximum likelihood estimation) se define como el producto de las probabilidades. Vamos a usar este número como indicador de cuan bueno es nuestro modelo. Así, si el estimador es 1 el modelo es perfecto. Sin embargo, noten que producto de valores entre 0 y 1 generalmente está mal condicionado numéricamente, es decir, da números muy chicos, más chicos que los límites de representación de números en punto flotante y se termina redondeando a cero. Para evitar esto se usa el logaritmo del likelihood. Además, recuerden que $$log(a*b*c) = log(a)+(log(b)+log(c)$$ 
Por lo tanto podemos calcular el log likelihood de la siguiente manera

In [ ]:
bigram_prob = bigram_count/bigram_count.sum(axis=1, keepdims=True)  # Calculamos la matriz de probabilidades (en lugar de matriz de cuentas)

In [ ]:
log_likelihood = 0.0
for d in dataset[:2]:
    example = ['*'] + list(d) + ['*']
    for c1, c2 in zip(example, example[1:]):
        prob = bigram_prob[ctoi[c1], ctoi[c2]]
        log_likelihood += np.log(prob)
        print(f'{c1}{c2} {prob:.4f}')
print(f'{log_likelihood=:0.4f}')

Para hacer comparaciones entre modelos y datasets, lo mejor normalizar el (log) likelihood por la cantidad de muestras utilizada para calcularlo. Así, usamos el promedio del (log) likelihood.

In [ ]:
prodprobs = 1
log_likelihood = 0.0
n = 0
for d in dataset[:2]:
    example = ['*'] + list(d) + ['*']
    for c1, c2 in zip(example, example[1:]):
        prob = bigram_prob[ctoi[c1], ctoi[c2]]
        prodprobs *= prob
        log_likelihood += np.log(prob)
        n += 1
        print(f'{c1}{c2} {prob:.4f}')
print(f'{prodprobs=} <== Noten que pequeño este número')  # <== Noten que pequeño este número
print(f'{log_likelihood= :0.4f}')
print(f'Average log_likelihood: {log_likelihood/n:0.4f}')

Por ultimo, vamos a usar esto como función de perdida (loss function). La semántica de la loss function indica que cuanto menor es el valor, mejor es el modelo. Por lo tanto, usamos el negative log likelihood. Como el logaritmo es una función monotónica creciente, que para las probabilidades va desde $-\infty$ hasta 0, el negativo de la función es monotónicamente descreciente desde $\infty$ hasta 0.

In [ ]:
x = np.arange(0,1.001, 0.001)
plt.plot(np.log(x), label='log(x)')
plt.plot(-np.log(x), label='-log(x)')
plt.grid()
plt.legend()

Se cumplen entonces las siguientes equicalencias:
- Maximizar el Likelihood es equivalente a:
- Maximizar el Log Likelihood que es equivalente a:
- Maximizar el Promedio del Log Likelihood que es equivalente a:
- Minimizar el Negativo del Promedio del Log Likelihood

Por lo tanto, podemos usar el negative log likelihood promedio como función de perdida y para evaluar que tan buenos son nuestros modelos. Esta función es la misma que usaremos para entrenar desde los modelos de redes neuronales hasta los transformers.

In [ ]:
def nll(s: list[str]) -> float:
    log_likelihood = 0.
    n = 0
    for d in s:
        example = ['*'] + list(d) + ['*']
        for c1, c2 in zip(example, example[1:]):
            prob = bigram_prob[ctoi[c1], ctoi[c2]]
            log_likelihood += np.log(prob)
            n += 1
            print(f'{c1}{c2} {prob:.4f} {np.log(prob):0.4f}')
    return -log_likelihood/n

In [ ]:
nll(['joaquin.cmm*']) # <-- Por esto necesitamos model smoothing..

# Model Smoothing